In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import t as tstats
from scipy.stats.mstats import winsorize
from scipy.stats import pearsonr
from scipy.stats import wilcoxon
import warnings
warnings.filterwarnings("ignore")

/databricks/python/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
df_au_eid = pd.read_csv(
    r"/dbfs/path/hbcu/hbcu_au_eid.csv"
)

df_eid = pd.read_csv(
    r"/dbfs/path/hbcu/hbcu_eid.csv"
)

df_attr = pd.read_csv(r"/dbfs/path/hbcu/full_hbcu.csv")

df_multiinst = pd.read_csv(r"/dbfs/path/hbcu/hbcu_multiinst_info.csv")



In [ ]:
# mark pubs in 1 and 2 stages

t = df_au_eid.query('year >= 2010 and year <= 2020').merge(df_multiinst, on=['PersonId'], how='inner')

def get_stage(row):
  vars = [row['year'], row['year_start1'], row['year_end1'], row['year_start2'], row['year_end2']]
  for i, var in enumerate(vars):
    if isinstance(var, str):
      vars[i] = 0

  year, year_start1, year_end1, year_start2, year_end2 = vars[0], vars[1], vars[2], vars[3], vars[4]
  if year <= year_end1 and year >= year_start1:
    return 0
  if year <= year_end2 and year >= year_start2:
    return 1
  else:
    return np.nan
  
t['stage'] = t.apply(lambda row: get_stage(row), axis=1)
t

,Unnamed: 0_x,eid,PersonId,auid,year,Authorseq,last_author_seq,publication_type,DegreeYear,Unnamed: 0_y,univ1,year_start1,year_end1,isHBCU1,univ2,year_start2,year_end2,isHBCU2,MoveType,period1_len,period2_len,stage
0,0,85041492438,245343,36077375900,2018,18,27,ar,2008,2564,Tennessee State University,2011,2017,1,NaN,NaN,NaN,NaN,S,7.0,NaN,NaN
1,1,85032862966,245343,36077375900,2017,1,2,ed,2008,2564,Tennessee State University,2011,2017,1,NaN,NaN,NaN,NaN,S,7.0,NaN,0.0
2,2,85044249714,245343,36077375900,2017,5,5,ar,2008,2564,Tennessee State University,2011,2017,1,NaN,NaN,NaN,NaN,S,7.0,NaN,0.0
3,3,79960449192,245343,36077375900,2011,5,5,ar,2008,2564,Tennessee State University,2011,2017,1,NaN,NaN,NaN,NaN,S,7.0,NaN,0.0
4,6,79953229600,245343,36077375900,2011,10,17,ar,2008,2564,Tennessee State University,2011,2017,1,NaN,NaN,NaN,NaN,S,7.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30912,33613,85057630733,523792,7003534893,2019,2,8,ar,1988,820,Florida A&M University,2013,2020,1,NaN,NaN,NaN,NaN,S,8.0,NaN,0.0
30913,33568,85032671402,260068,7102072429,2018,3,3,ar,1988,3162,Texas Southern University,2011,2017,1,NaN,NaN,NaN,NaN,S,7.0,NaN,NaN
30914,33570,84954414968,260068,7102072429,2016,2,2,ar,1988,3162,Texas Southern University,2011,2017,1,NaN,NaN,NaN,NaN,S,7.0,NaN,0.0
30915,33578,85030182490,260068,7102072429,2018,1,3,ar,1988,3162,Texas Southern University,2011,2017,1,NaN,NaN,NaN,NaN,S,7.0,NaN,NaN


In [ ]:
# confidence interval
def calculate_confidence_interval(group):
    std = group.std()
    n = len(group)
    conf_level = 0.95
    dof = n - 1
    t_value = tstats.ppf((1 + conf_level) / 2, dof)
    se = std / np.sqrt(n)
    margin_error = t_value * se
    return margin_error


def winsorize_group(group):
    return winsorize(group, limits=[0, 0.05])


In [ ]:
import plotly.colors as pcolors

def generate_color_list(num_colors):
    # Get a list of colors from the Plotly color scales
    color_scale = pcolors.DEFAULT_PLOTLY_COLORS
    colors = color_scale * (num_colors // len(color_scale) + 1)

    color_list = []
    # Modify the colors to include the desired opacity
    for i in range(num_colors):
        color_list.append(f"rgba{colors[i][3:-1]}" + ", {})")
    return color_list

def generate_symbol_list(num_symbols):
    # Define a set of symbols to use
    symbols = ['circle', 'square', 'diamond', 'cross', 'x', 'star', 'triangle-up', 'triangle-down']

    # Repeat the symbols to match the desired number
    symbol_list = symbols * (num_symbols // len(symbols) + 1)
    return symbol_list[:num_symbols]

# Example usage
num_items = 5
colors = generate_color_list(num_items)
symbols = generate_symbol_list(num_items)

# Print the generated lists
print(colors)
print(symbols)


['rgba(31, 119, 180, {})', 'rgba(255, 127, 14, {})', 'rgba(44, 160, 44, {})', 'rgba(214, 39, 40, {})', 'rgba(148, 103, 189, {})']
['circle', 'square', 'diamond', 'cross', 'x']


In [ ]:
def plot_styling(
    fig,
    size=(250, 270),
    title=None,
    xtitle="Year (first post-tenure year=1)",
    dtick=None,
):
    layout_params = dict(
      title=dict(text=title, x=0.5, xanchor='center', xref='container', font=dict(size=14)),
        width=size[1],
        height=size[0],
        template='simple_white',
        font=dict(size=11, family="Arial"),
        margin=dict(l=50, r=50, b=50, t=50),
    )

    fig.update_layout(
        **layout_params
    )

    fig.update_xaxes(title=dict(text=xtitle, standoff=0), dtick=dtick)

    return fig

In [ ]:
def add_lines_with_errorband(fig, x, y, lower, upper, name, color, showlegend=False, showband=True, dash=None, **kwargs):
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode="lines",
            name=name,
            line=dict(color=colors[color].format(1), width=2, dash=dash),
            showlegend=showlegend,
        ),**kwargs,
    )

    # add error band
    if showband:
        x_error = np.concatenate((x, x[::-1]))
        y_error = np.concatenate((lower, upper[::-1]))
        fig.add_trace(
            go.Scatter(
                x=x_error,
                y=y_error,
                fill="toself",
                fillcolor=colors[color].format(0.2),
                line=dict(color="rgba(255,255,255,0)"),
                showlegend=False,
            ),**kwargs,
        )

    return fig

# prob plots

In [ ]:
# based table

df_base = (
    t[
        [
            "PersonId",
            "year_start2",
            "MoveType",
        ]
    ]
    .merge(
        df_au_eid[["PersonId", "eid", "year", "Authorseq", "last_author_seq"]],
        on=["PersonId"],
    )
    .merge(
        df_eid[
            [
                "PersonId",
                "eid",
                "subfield_hybrid",
                "n_references",
                "avgnum_past_eid",
                "publication_type",
            ]
        ],
        on=["PersonId", "eid"],
    )
        .assign(tau=lambda t: t["year"] - t["year_start2"])
    .drop_duplicates()
)

tau_range = np.arange(-5, 6, 1)

In [ ]:
def fill_nan_with_zero(val, ind, col, t_range, person):
    if pd.isna(val):
        person_id = ind
        tau_range = t_range[t_range[person] == person_id]
        if not tau_range.empty:
            min_tau = tau_range['min_tau'].values[0]
            max_tau = tau_range['max_tau'].values[0]
            if min_tau <= col <= max_tau:
                return 0
    return val

In [ ]:
# for each df, construct avg pub by tau
def get_mean_pub_bytau(t, is_treat=True):
    person = 'PersonId' if is_treat else 'index' # index for associate professor with replacement
    t = t[[groupvar, person, "tau", "year_start2", "eid"]].drop_duplicates()

    # get min and max tau
    if is_treat:
        t["min_tau"] = t["year_start2"].apply(lambda x: max(tau_range[0], 2006 - x))
    else:
        t["min_tau"] = t["year_start2"].apply(lambda x: max(tau_range[0], 2011 - x))
    t["max_tau"] = t["year_start2"].apply(lambda x: min(tau_range[-1], 2020 - x))
    t_range = t[[person, "min_tau", "max_tau"]].drop_duplicates()

    df_cite_norm = (
        t.groupby([groupvar, person, "tau"])
        .agg(mean1=("eid", "nunique"))
        .reset_index()
        .query("tau in @tau_range")
        .pivot(columns=["tau"], values=["mean1"], index=[groupvar, person])
    )
    # fill 0 for tau range
    for ind, row in df_cite_norm.iterrows():
        for col in df_cite_norm.columns:
            val = row[col]
            row[col] = fill_nan_with_zero(val, ind[-1], col[-1], t_range, person)

    groups = df_cite_norm.stack().reset_index().dropna().groupby([groupvar, "tau"])

    if is_treat:
        df_cite_norm = groups.agg(
            mean2=("mean1", "mean"),
            confint=("mean1", calculate_confidence_interval),
        ).reset_index()
    else:
        df_cite_norm = groups.agg(
            mean2=("mean1", "mean"),
        ).reset_index()
        df_cite_norm['confint'] = np.nan

    return df_cite_norm

In [ ]:
groupvar = 'all'
df_norms = []

for movetype in ['M-HBCU-PWI', 'M-PWI-HBCU']:
  t = df_base.query("MoveType	== @movetype")[["PersonId", "tau", "eid", "year_start2"]].drop_duplicates()
  print(t['PersonId'].nunique())
  t['all'] = 1

  df_norm = get_mean_pub_bytau(t, is_treat=True)
  df_norms.append(df_norm)

df_norms[0]

49
60


,all,tau,mean2,confint
0,1,-5.0,2.285714,1.343412
1,1,-4.0,2.265306,1.153079
2,1,-3.0,2.408163,0.948883
3,1,-2.0,3.877551,1.796294
4,1,-1.0,3.163265,1.234529
5,1,0.0,3.612245,1.463220
6,1,1.0,3.808511,1.527659
7,1,2.0,4.282609,1.620092
8,1,3.0,5.558140,2.351763
9,1,4.0,4.567568,1.640081


In [ ]:
# plot
for df_norm in df_norms:
  fig = go.Figure()
  for i, domain in enumerate([1]):
      t1_domain = df_norm.query(f"{groupvar} == @domain")
      x = tau_range[:]
      y1 = t1_domain["mean2"].to_numpy()
      y1_upper, y1_lower = (
          t1_domain["mean2"].to_numpy() + t1_domain["confint"].to_numpy(),
          t1_domain["mean2"].to_numpy() - t1_domain["confint"].to_numpy(),
      )

      fig = add_lines_with_errorband(
          fig, x, y1, upper=y1_upper, lower=y1_lower, name=domain, color=i, showlegend=True
      )

  fig = plot_styling(fig, size=(250, 300), title=None)
  fig.update_yaxes(
      title=dict(text="# papers"),
      titlefont=dict(color="black"),
      tickfont=dict(color="black"),
  )
  # legend
  fig.update_layout(
      showlegend=True,
      legend=dict(x=1, y=1, xanchor="left", yanchor="top"),
  )

  # mark 0-1 zone
  fig.add_vrect(x0=0, x1=1, line_width=0, fillcolor="gray", opacity=0.2)

  fig.show()

In [ ]:
# citation

In [ ]:
metrics = ["arc_3yr_noself", jif_metric]

for movetype in ['M-HBCU-PWI', 'M-PWI-HBCU']:
  print(movetype)
  t = (
      df_base.query("MoveType	== @movetype").merge(
          df_eid[["eid", "arc_3yr_noself"]],
          on=["eid"],
      )
      .fillna({"arc_3yr_noself": 0})
      .drop_duplicates()
  )

  for method in ['mean', 'max', 'min']:
    for i, metric in enumerate(metrics):
        df_norm = (
            ts[i]
            .groupby([groupvar, "PersonId", "tau"])
            .agg(mean1=(metric, method))
            .reset_index()
            .query("tau in @tau_range")
            .groupby([groupvar, "tau"])
            .agg(
                mean2=("mean1", "mean"),
                confint=("mean1", calculate_confidence_interval),
            )
            .reset_index()
        )
        df_norm['method'] = method
        df_norm['metric'] = metric
        df_norms.append(df_norm)

df_norms = pd.concat(df_norms)

M-HBCU-PWI
49 2182


M-PWI-HBCU
60 1591


In [ ]:
# regression

def segment_reg(
    df_tau_range,
    depvar,
    family="ols", # "poisson", "negbin", "logit", "gaussian"
    control=[],
    fix="PersonId Field_English".split(),
):
    # convert tau column to int
    df_tau_range["tau"] = df_tau_range["tau"].astype(int)
    # create dummy variables for each tau
    t = pd.concat(
        [df_tau_range, pd.get_dummies(df_tau_range["tau"], prefix="tau")], axis=1
    )
    t = t.drop(["tau_-5"], axis=1, errors='ignore')
    # replace - in column names with m
    t.columns = t.columns.str.replace("-", "m")
    tau_cols = [col for col in t.columns if col.startswith("tau_")]
    
    # convert to R dataframe
    pandas2ri.activate()
    r_dataframe = pandas2ri.py2rpy(t)

    if len(control) > 0:
      formula = f"{depvar} ~ {' + '.join(tau_cols)} + {' + '.join(control)} | {' + '.join(fix)}"
    else:
      formula = f"{depvar} ~ {' + '.join(tau_cols)} | {' + '.join(fix)}"
    
    print(formula)
    if family == 'ols':
      model = fixest.feols(Formula(formula), data=r_dataframe, cluster='~PersonId') #
    else:
      model = fixest.femlm(Formula(formula), data=r_dataframe, family=family)
    # Extract the coefficients, p-values, and confidence intervals
    table = fixest.esttable(model, coefstat='confint', signif_code=np.NaN) #,keep='tau'
    # convert to pandas
    pd_table = pandas2ri.rpy2py(table)
    pd_table.columns = ['var', 'value']
    print(pd_table)
    pd_table = pd_table['value'] #.query('var in @tau_cols')
    pd_table = pd_table.str.extract(r'([-0-9.e]+)\s+\[([-0-9.e]+);\s+([-0-9.e]+)\]').applymap(lambda x: float(x))
    pd_table.columns = ['beta', 'lower', 'upper']
    df_beta, df_conf = pd_table['beta'].to_numpy(), pd_table[['lower', 'upper']].to_numpy()
    return df_beta, df_conf

In [ ]:
# productivity

for movetype in ['M-HBCU-PWI', 'M-PWI-HBCU']:
  print(movetype)
  t = df_base.query("MoveType	== @movetype")[["PersonId", "tau", "eid"]].drop_duplicates()

  df_tau_range = (
      t.groupby(["PersonId", "tau"])
      .agg(mean1=("eid", "nunique"))
      .reset_index()
      .query("tau in @tau_range")
      # fill 0
      .pivot(columns=['tau'], values=['mean1'], index=["PersonId"])
      .fillna(0)
      .stack()
      .reset_index()
  )

  depvar = 'mean1'
  df_beta, df_conf = segment_reg(
      df_tau_range,
      depvar,
      family="poisson", # "poisson", "negbin", "logit", "gaussian"
      control=[],
      fix="PersonId".split(),
  )




M-HBCU-PWI
mean1 ~ tau_m4 + tau_m3 + tau_m2 + tau_m1 + tau_0 + tau_1 + tau_2 + tau_3 + tau_4 + tau_5 | PersonId
                var                      value
1   Dependent Var.:                      mean1
2                                             
3            tau_m4  -0.0090 [-0.4483; 0.4304]
4            tau_m3   0.0522 [-0.4081; 0.5125]
5            tau_m2    0.5285 [-0.0441; 1.101]
6            tau_m1   0.3249 [-0.1985; 0.8484]
7             tau_0    0.4577 [-0.0868; 1.002]
8             tau_1    0.4689 [-0.1457; 1.083]
9             tau_2     0.5647 [0.0227; 1.107]
10            tau_3     0.7580 [0.1951; 1.321]
11            tau_4   0.4114 [-0.1223; 0.9451]
12            tau_5   0.0855 [-0.5990; 0.7701]
13   Fixed-Effects:  -------------------------
14         PersonId                        Yes
15  _______________  _________________________
16  VCOV: Clustered               by: PersonId
17     Observations                        539
18     Squared Cor.                    0.6

In [ ]:
df_eid

,Unnamed: 0,eid,PersonId,total_cite_noself,3yr_cite_noself,5yr_cite_noself,arc_all_noself,arc_3yr_noself,arc_5yr_noself,subfield_hybrid,avgnum_past_eid,time,publication_type,n_au,n_references
0,0,3342995569,133250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,2014-06-01,ch,1,9
1,1,20444455784,133250,3.0,1.0,1.0,0.352957,0.566096,0.267457,criminology,17.0,2014-06-01,ch,1,84
2,2,78650391236,754316,57.0,17.0,30.0,1.415585,2.153325,2.007476,nanoscience & nanotechnology,8.5,2011-01-01,ar,7,41
3,3,78650610372,77600,35.0,14.0,21.0,2.870587,6.266766,5.063650,design practice & management,21.5,2011-01-01,ar,3,32
4,4,78650610372,105512,35.0,14.0,21.0,2.870587,6.266766,5.063650,design practice & management,21.5,2011-01-01,ar,3,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33527,33527,85159447121,103321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2023-04-01,pp,4,0
33528,33528,85159564782,750776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2023-03-01,pp,22,0
33529,33529,85163480768,62516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.5,2023-01-01,ar,4,0
33530,33530,85164663758,122157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0,2023-01-01,re,1,0


In [ ]:
# citation

for movetype in ['M-HBCU-PWI', 'M-PWI-HBCU']:
  t = (
      df_base.query("MoveType	== @movetype").merge(
          df_eid[["eid", "arc_3yr_noself", 'n_au']],
          on=["eid"],
      )
      .query("tau in @tau_range")
      .fillna({"arc_3yr_noself": 0})
      .drop_duplicates()
  )

  t['arc_3yr_noself'] = t.groupby(['tau'])['arc_3yr_noself'].transform(winsorize_group)

  depvar = 'arc_3yr_noself'
  df_beta, df_conf = segment_reg(
      t,
      depvar,
      family="ols", # "poisson", "negbin", "logit", "gaussian"
      control=['avgnum_past_eid', 'n_au', 'n_references'],
      fix="PersonId publication_type subfield_hybrid".split(),
  )




arc_3yr_noself ~ tau_m4 + tau_m3 + tau_m2 + tau_m1 + tau_0 + tau_1 + tau_2 + tau_3 + tau_4 + tau_5 + avgnum_past_eid + n_au + n_references | PersonId + publication_type + subfield_hybrid
                 var                      value
1    Dependent Var.:             arc_3yr_noself
2                                              
3             tau_m4   0.0843 [-0.6594; 0.8280]
4             tau_m3  -0.1897 [-0.9950; 0.6155]
5             tau_m2   0.0461 [-0.7202; 0.8123]
6             tau_m1   0.0035 [-0.8640; 0.8710]
7              tau_0  -0.1182 [-0.6852; 0.4488]
8              tau_1    0.9619 [-0.5922; 2.516]
9              tau_2   0.0354 [-0.8410; 0.9119]
10             tau_3    0.1779 [-0.9418; 1.298]
11             tau_4     0.2100 [-1.002; 1.422]
12             tau_5    0.1562 [-0.8394; 1.152]
13   avgnum_past_eid   0.0023 [-0.0014; 0.0061]
14              n_au   0.0440 [-0.0149; 0.1030]
15      n_references  0.0109 [-5.98e-5; 0.0219]
16    Fixed-Effects:  ----------------------